In [1]:
import pandas as pd
import PyPDF2

In [3]:
pdfFileObj = open('dab_106.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [4]:
# create page object and extract text
pageObj = pdfReader.getPage(3)
page1 = pageObj.extractText()
page_blob = pageObj.extractText()


In [5]:
output = ""
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    output += pageObj.extractText()

In [6]:
page1 = output

In [7]:
page1

'Course ID: 030277 Academic Year: 2021/22\nCourse Description:\nThis course provides students with a high-level overview of the broad field of Artificial\nIntelligence (AI). Students will explore some basic definitions and concepts and will learn to\ndistinguish between the various branches that make up AI.\xa0 This course outlines the history of AI,\nfocusing on major developments and reasons behind its current resurgence.\xa0 The bulk of the course\nwill be spent on understanding the core elements of AI systems.\nPre-Requisites: Category: Vocational None\nCo-Requisites: Course Credits: 3.00 None\nSpecial Conditions: Academic Level: Credit (Post Sec) None\nInstructional Hours: Classroom Instruction 30\nLaboratory/Workshops 15\nOther 0\nTotal Hours 45\nAcademic Department:\nWindsor: Zekelman School of Information Technology\n      Chair:\nChatham:\n      Chair:\nRevised By: Harpreet Virk\nLast Revision: 2022/01/20\nCourse Outline\nDAB 106 Introduction to Artificial Intelligence\nPage  

In [8]:
course_name = page1.replace('\n \n',', ').replace('\n','')
course_name = course_name.split('Course Outline')
course_name = course_name[1].split('Page  1')
course_name = course_name[0]
course_name

'DAB 106 Introduction to Artificial Intelligence'

In [9]:
# strip away page header
page1 = page1[25:]

# insert commas to separate variables and then remove excess strings
page1 = page1.replace('\n \n',', ').replace('\n','')


In [10]:
page1 = page1.split(',')


In [11]:
s = pd.Series(page1)
out = (s.str.extractall(r'(?P<Object>Assignments|Exam 1|Exam 2|Exam 3|Exam|Project|Quizzes|Final Exam)|(?P<Weight>\d+.00)|(?P<Frequency>Frequency: \d+)')
       .groupby(level=0).first()
       )
df = pd.DataFrame(out)
df

Object Weight     Frequency
1           None   3.00          None
25          Exam   None          None
71   Assignments  20.00          None
72          None   None  Frequency: 4
83        Exam 1  25.00          None
84          None   None  Frequency: 1
91    Final Exam  25.00          None
92          None   None  Frequency: 1
98       Project  20.00          None
99          None   None  Frequency: 2
107      Quizzes  10.00          None
108         None   None  Frequency: 5

In [12]:
'''out = (pd.Series(page1)
       .str.extract(r'(?P<Weight>\d+.00%)\W*(?P<Object>\w+)')
       .dropna(subset='Object')
      )

print(out)'''

"out = (pd.Series(page1)\n       .str.extract(r'(?P<Weight>\\d+.00%)\\W*(?P<Object>\\w+)')\n       .dropna(subset='Object')\n      )\n\nprint(out)"

In [13]:
df['Frequency'] = df.Frequency.shift(-1)
#df

In [14]:
df = df.dropna()
#df

In [15]:
df

Object Weight     Frequency
71   Assignments  20.00  Frequency: 4
83        Exam 1  25.00  Frequency: 1
91    Final Exam  25.00  Frequency: 1
98       Project  20.00  Frequency: 2
107      Quizzes  10.00  Frequency: 5

In [16]:
df['Frequency'] = df['Frequency'].str.extract('(\d+)').astype(int)
df['Weight'] = df['Weight'].astype(float)
#df

C:\Users\rites\AppData\Local\Temp\ipykernel_9172\1332641898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Frequency'] = df['Frequency'].str.extract('(\d+)').astype(int)
C:\Users\rites\AppData\Local\Temp\ipykernel_9172\1332641898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weight'] = df['Weight'].astype(float)


In [17]:
df['Weight per Object'] = df.Weight/df.Frequency

C:\Users\rites\AppData\Local\Temp\ipykernel_9172\3609157279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weight per Object'] = df.Weight/df.Frequency


In [38]:
Exam = "Exam is less than 25% or more than 40%"
Quize = "Quiz weightage greater than 5%"
Test = "Test is less than 5% or more than 24%"
cols = ['Exam', 'Quize', 'Test']

In [39]:
Rule_Violation = {"c1":cols,"Rules":[Exam,Quize,Test]}

In [40]:
Rule_Violation

{'c1': ['Exam', 'Quize', 'Test'],
 'Rules': ['Exam is less than 25% or more than 40%',
  'Quiz weightage greater than 5%',
  'Test is less than 5% or more than 24%']}

In [41]:
Rule_Violation = pd.DataFrame(Rule_Violation)
Rule_Violation

c1                                   Rules
0   Exam  Exam is less than 25% or more than 40%
1  Quize          Quiz weightage greater than 5%
2   Test   Test is less than 5% or more than 24%

In [42]:
df['Flag'] = ((df['Object'].str.contains('Quiz', regex=True)) & (abs(df['Weight per Object']) <= 4)) | ((df['Object'].str.contains('Exam', regex=True)) & (df['Weight per Object'] >= 25) & (df['Weight per Object'] <= 40)) | ((df['Object'].str.contains('Test', regex=True)) & (df['Weight per Object'] >= 5) & (df['Weight per Object'] <= 24))
df

C:\Users\rites\AppData\Local\Temp\ipykernel_9172\834351203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flag'] = ((df['Object'].str.contains('Quiz', regex=True)) & (abs(df['Weight per Object']) <= 4)) | ((df['Object'].str.contains('Exam', regex=True)) & (df['Weight per Object'] >= 25) & (df['Weight per Object'] <= 40)) | ((df['Object'].str.contains('Test', regex=True)) & (df['Weight per Object'] >= 5) & (df['Weight per Object'] <= 24))


Object  Weight  Frequency  Weight per Object   Flag
71   Assignments    20.0          4                5.0  False
83        Exam 1    25.0          1               25.0   True
91    Final Exam    25.0          1               25.0   True
98       Project    20.0          2               10.0  False
107      Quizzes    10.0          5                2.0   True

In [63]:
Rule_Violation[Rule_Violation['c1']=='Exam']['Rules'].to_string(index=False)

'Exam is less than 25% or more than 40%'

In [64]:
list = []
for i in df['Object']:
    if 'Exam' in i:
        list.append(Rule_Violation[Rule_Violation['c1']=='Exam']['Rules'].to_string(index=False))
    elif 'Quize' in i:
        list.append(Rule_Violation[Rule_Violation['c1']=='Quize']['Rules'].to_string(index=False))
    elif 'Test' in i:
        list.append(Rule_Violation[Rule_Violation['c1']=='Test']['Rules'].to_string(index=False))
    else:
        list.append(0)

In [66]:
df['Violation'] = list

C:\Users\rites\AppData\Local\Temp\ipykernel_9172\3280556087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Violation'] = list


In [67]:
df

Object  Weight  Frequency  Weight per Object   Flag  \
71   Assignments    20.0          4                5.0  False   
83        Exam 1    25.0          1               25.0   True   
91    Final Exam    25.0          1               25.0   True   
98       Project    20.0          2               10.0  False   
107      Quizzes    10.0          5                2.0   True   

                                  Violation  
71                                        0  
83   Exam is less than 25% or more than 40%  
91   Exam is less than 25% or more than 40%  
98                                        0  
107                                       0

In [17]:
df['Desc'] = str(df['Weight']) + str("Hello")

C:\Users\rites\AppData\Local\Temp\ipykernel_6916\848595481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Desc'] = str(df['Weight']) + str("Hello")


In [15]:
#print(df[df['Flag']==False])

In [16]:
output = ""
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    output += pageObj.extractText()
page1 = output

In [17]:
page1 = page1.replace('\n \n \n•','>>>').replace(')\n \n•', '>>>').replace('\n \n•', '>').replace('\n•','>').replace('\nEKS:', '')

In [18]:
page1 = page1.split('>>>')
#page1

In [19]:
del page1[0]
#page1

In [21]:
clo1 = page1[0].split('>')
#clo1

In [31]:
clo1[0]

'Examine the key components of Artificial Intelligence (AI) systems (CLO #1)'

In [22]:
sep = ' '
clo1_verbs = []
for i in range(len(clo1)):
    x = clo1[i].split(sep, 1)[0]
    clo1_verbs.append(x)

In [23]:
word_list = pd.read_excel('word_list.xlsx')
#word_list.head()

In [24]:
scores = []

for i in range(len(clo1_verbs)):
    if word_list['1'].str.contains(clo1_verbs[i]).any():
        scores.append(1)
    elif word_list['2'].str.contains(clo1_verbs[i]).any():
        scores.append(2)
    elif word_list['3'].str.contains(clo1_verbs[i]).any():
        scores.append(3)
    elif word_list['4'].str.contains(clo1_verbs[i]).any():
        scores.append(4)
    elif word_list['5'].str.contains(clo1_verbs[i]).any():
        scores.append(5)
    elif word_list['6'].str.contains(clo1_verbs[i]).any():
        scores.append(6)
    else:
        scores.append(0)

#print(scores)

In [25]:
clo1_df = {'Verbs': clo1_verbs, 'Scores': scores}
clo1_df = pd.DataFrame(clo1_df)
clo1_df

Verbs  Scores
0   Examine       4
1   Provide       0
2   Explain       2
3   Discuss       2
4   Discuss       2
5  Describe       1

In [87]:
clo1_df['Dist'] = abs(clo1_df['Scores'][0] - clo1_df['Scores'])
clo1_df

Verbs  Scores  Dist
0   Examine       4     0
1   Provide       0     4
2   Explain       2     2
3   Discuss       2     2
4   Discuss       2     2
5  Describe       1     3

In [90]:
bloom_violations = clo1_df[clo1_df['Dist'] >= 2][1:]

In [92]:
bloom_violations[['Verbs', 'Dist']][1:]

Verbs  Dist
3   Discuss     2
4   Discuss     2
5  Describe     3

In [27]:
print("Word which does not exist in Bloom's Conginitive Domain list:")
print(clo1_df[clo1_df['Scores'] == 0])

Word which does not exist in Bloom's Conginitive Domain list:
     Verbs  Scores
1  Provide       0


In [28]:
print("Assessments Violations")
print(df[df['Flag']==False])

Assessments Violations
         Object  Weight  Frequency  Weight per Object   Flag
71  Assignments    20.0          4                5.0  False
98      Project    20.0          2               10.0  False


In [29]:
df[['Object', 'Flag']]

Object   Flag
71   Assignments  False
83          Exam   True
91    Final Exam   True
98       Project  False
107      Quizzes   True

In [74]:
rules = pd.read_excel('rules.xlsx', index_col='Rule No.')

In [75]:
rules

Assessmet Validation Rule
Rule No.                                                   
1         An assessment value between 1% and 4% should b...
2         An assessment value between 5% and 24% should ...
3         An assessment value between 25% and 40% should...